In [ ]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# installing requirements
!pip install imageio==2.19.3
!pip install opencv-python==4.6.0.66
!pip install tensorboardX==2.2
!pip install einops==0.4.1
!pip install scikit-learn==1.1.1
!pip install matplotlib==3.5.1
!pip install protobuf==3.20.0
!pip install fire==0.4.0
!pip install moviepy==1.0.3

In [130]:
import sys
sys.path.append('/content/drive/My Drive/drive_folder/pips/')
import time
import numpy as np
import io
import os
from PIL import Image
import cv2
import saverloader
from nets.pips import Pips
import utils.improc
import random
import glob
from utils.basic import print_, print_stats
import torch
from tensorboardX import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gc
import imageio.v2 as imageio

# save numpy array as npz file
from numpy import asarray
from numpy import savez_compressed

In [ ]:
# unziping frames into data folder
!$data_dir="frames_small"
!unzip -d "$data_dir"/ /content/drive/MyDrive/"$data_dir".zip

In [110]:
#create a folder to store training data in
!mkdir training_data

In [95]:
# specify the name of the data folder
data_dir = "frames_small"

In [ ]:
# downloading their reference model
!bash /content/drive/MyDrive/drive_folder/pips/get_reference_model.sh

In [2]:
def create_model(model_weights):
  model = Pips(stride=4).cuda()
  parameters = list(model.parameters())
  _ = saverloader.load(model_weights, model)
  global_step = 0
  model.eval()
  return model

In [97]:
def run_model(model, rgbs, N, split):
    rgbs = rgbs.cuda().float() # B, S, C, H, W

    B, S, C, H, W = rgbs.shape
    rgbs_ = rgbs.reshape(B*S, C, H, W)
    H_, W_ = 360, 640
    rgbs_ = F.interpolate(rgbs_, (H_, W_), mode='bilinear')
    H, W = H_, W_
    rgbs = rgbs_.reshape(B, S, C, H, W)

    # pick N points to track; we'll use a uniform grid
    N_ = np.sqrt(N).round().astype(np.int32)
    grid_y, grid_x = utils.basic.meshgrid2d(B, N_, N_, stack=False, norm=False, device='cuda')
    grid_y = 8 + grid_y.reshape(B, -1)/float(N_-1) * (H-16)
    grid_x = 8 + grid_x.reshape(B, -1)/float(N_-1) * (W-16)
    xy = torch.stack([grid_x, grid_y], dim=-1) # B, N_*N_, 2
    _, S, C, H, W = rgbs.shape

    print_stats('rgbs', rgbs)

    # splitting grid of points into 100 tensors along dim=1
    if split:
       xy_split = torch.split(xy, 100, dim=1)
       preds_split = []

      # for each splitted point compute trajs
       for i in range(len(xy_split)):      
         preds, preds_anim, vis_e, stats = model(xy_split[i], rgbs, iters=6)
         preds_split.append(preds[-1])

       # put trajs back together
       trajs_e = torch.cat(preds_split, 2)
       

    else:
      preds, preds_anim, vis_e, stats = model(xy, rgbs, iters=6)
      # preds is a list of torch tensors 
      trajs_e = preds[-1] # tensor of shape (no.frames, no.tracking points, 2)
    print_stats('trajs_e', trajs_e)

    # return vis?? as well
    return trajs_e

In [127]:
def generate_training_data(pips_model, video_name, n, split):

    model = pips_model
    ## choose hyps
    B = 1
    S = 8
    N = n**2 # number of points to track

    # get frames of a file
    filenames = glob.glob(data_dir+'/content/frames/'+video_name+'/frames/*.jpg')
    filenames = sorted(filenames)
    max_iters = len(filenames)//S # run each unique subsequence

    global_step = 0

    # empty array to store training data
    a = np.empty([1,2])

    # run model each of 8 frames
    while global_step < max_iters:
        global_step += 1
        print("step",global_step,"out of",max_iters)

        try:
            rgbs = []
            for s in range(S):
                fn = filenames[(global_step-1)*S+s]
                if s==0:
                    print('start frame', fn)
                im = imageio.imread(fn)
                im = im.astype(np.uint8)
                rgbs.append(torch.from_numpy(im).permute(2,0,1))
            rgbs = torch.stack(rgbs, dim=0).unsqueeze(0) # 1, S, C, H, W

            with torch.no_grad():
                trajs_e = run_model(model, rgbs, N, split)

            # store training_img (frame 1 and 8) and trajs_e
            rgb_array1 = rgbs[0][0].permute(1, 2, 0).numpy() # from tensor to numpy array
            rgb_array8 = rgbs[0][-1].permute(1, 2, 0).numpy() # from tensor to numpy array
            training_img1 = cv2.resize(rgb_array1 , (240, 120)) # resizing image
            training_img8 = cv2.resize(rgb_array8 , (240, 120)) # resizing image
            trajs_array = trajs_e.cpu().numpy()

            # store training data
            if global_step ==1:
              a = np.array([(training_img1, training_img8, trajs_array[0][-1])], dtype=object)
            else:
              a = np.append(a, [(training_img1, training_img8, trajs_array[0][-1])],axis = 0)

        except FileNotFoundError as e:
            print('error', e)

    # save traning data to npz file
    savez_compressed("training_data/"+video_name+'.npz', a)


In [ ]:
model = create_model('reference_model')

In [125]:
# create a list of file names to go through
videos_list = [f for f in os.listdir('frames_small/content/frames/') if not f.startswith('.')]
print(videos_list)

['Tai chiCurso Chi Kung de los Seis Sonidos Curativos-PvjYVsRK4Dg', 'Tai chi10 forms 2009-KX-dEeB47sc', 'Tai chiShaolin Basics Are From Theater!-gyms4lomW50', 'Tai chiRen Guang Yi - Chen Style Taiji silk reeling part II-Wh3AmDsdQtM', 'Tai chi熊門楊家太極拳111式第一段\u3000李國光老師示範-X_9SJZuSWQU', 'Tai chiCanda - Tai Chi Chuan Yang-Stil - Sanfte Bewegungsformen für Einsteiger-f7NkWPgh1-o', 'Tai chiWee Kee Jin - Keeping Your Structure-7jn9jeAbChE', 'Tai chiYang Family Tai Chi q&a Knee Brush-WPeVwAhTNuU', 'Tai chi陈式太极拳五十六式-vB8XTJfV4rY', 'Tai chi56式夕陽美功夫扇-uOw-z7CR7x8']


In [ ]:
# iterate through the videos list and generate training data files
for x in range(len(videos_list)):
  print("Now generating training data for {}, {} out of {}.".format(videos_list[x], x, len(videos_list)))
  generate_training_data(model, videos_list[x], 100, split=True)

In [ ]:
# zip all numpy files
!zip -r /content/training_data.zip /content/training_data
# cp NumPy zip file into drive
!cp training_data.zip /content/drive/MyDrive/

In [ ]:
# see training data of one video for verification
from numpy import load
np.set_printoptions(threshold=sys.maxsize)
data = load('Tai chiRen Guang Yi - Chen Style Taiji silk reeling part II-Wh3AmDsdQtM.npz', allow_pickle=True)
lst = data.files
for item in lst:
  for x in range(len(item)):
    plt.imshow(data[item][x][0])
    plt.show()
    plt.imshow(data[item][x][1])
    plt.show()
    print(data[item][x][2])